In [ ]:
import numpy as np
import cv2
import imutils
from matplotlib import pyplot as plt
DEBUG =1

img = cv2.imread('../data/s1_chana.jpg')
img = imutils.resize(img,width =400)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
if DEBUG: cv2.imshow('thresh',thresh)
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
if DEBUG: cv2.imshow('opening',opening)
# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)
if DEBUG: cv2.imshow('sure_bg',sure_bg)
# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
#if DEBUG: cv2.imshow('dist_transform',dist_transform)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
if DEBUG: cv2.imshow('sure_fg',sure_fg)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)
if DEBUG: cv2.imshow('sure_fg1',sure_fg)
    
# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)
if DEBUG: cv2.imshow('markers1',markers)

# Add one to all labels so that sure background is not 0, but 1
markers = markers+1

# Now, mark the region of unknown with zero
markers[unknown==255] = 0
if DEBUG: cv2.imshow('markers2',markers)
    
markers = cv2.watershed(img,markers)
img[markers == -1] = [0,255,0]
if DEBUG: cv2.imshow('img1',img)
cv2.imwrite('../data/water_mix.jpg',img)
cv2.waitKey(0)